In [ ]:
# -*- coding: utf-8 -*-
"""
TCC - ANÁLISE ESPAÇO-TEMPORAL DE ÁREAS VERDES EM SALVADOR
Notebook 100% AUTÔNOMO - RODA EM QUALQUER COLAB
"""

# ============================================================================
# 0. CONFIGURAÇÃO AUTOMÁTICA
# ============================================================================

print("Configurando ambiente...")

# Instalar pacotes
!pip install -q earthengine-api geemap geopandas plotly prophet libpysal esda splot mapclassify folium

# Imports
import ee, geemap, geopandas as gpd, pandas as pd, numpy as np, matplotlib.pyplot as plt
import plotly.express as px, plotly.graph_objects as go
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from libpysal.weights import Queen, KNN
from esda.moran import Moran, Moran_Local
from splot.esda import moran_scatterplot, lisa_cluster
import mapclassify, folium, json, warnings, os, urllib.request, zipfile
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

print("Bibliotecas instaladas")

# ============================================================================
# 1. AUTENTICAR NO GOOGLE EARTH ENGINE
# ============================================================================

try:
    ee.Initialize()
    print("GEE já autenticado")
except:
    print("Autenticando no Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize()
    print("GEE autenticado com sucesso!")

# ============================================================================
# 2. BAIXAR SHAPEFILE DO GITHUB
# ============================================================================

def baixar_shapefile():
    url = "https://raw.githubusercontent.com/seu_usuario/tcc-ndvi-salvador/main/data/bairros_final.zip"
    zip_path = "bairros_final.zip"
    extract_path = "data/"

    if not os.path.exists("data/bairros_final.shp"):
        print("Baixando shapefile do GitHub...")
        urllib.request.urlretrieve(url, zip_path)
        os.makedirs(extract_path, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(extract_path)
        os.remove(zip_path)
        print("Shapefile baixado!")
    else:
        print("Shapefile já existe")

baixar_shapefile()

# ============================================================================
# 3. CARREGAR SHAPEFILE
# ============================================================================

shapefile_path = "data/bairros_final.shp"
try:
    gdf_bairros = gpd.read_file(shapefile_path)
    if gdf_bairros.crs != "EPSG:4326":
        gdf_bairros = gdf_bairros.to_crs("EPSG:4326")
    print(f"Shapefile carregado: {len(gdf_bairros)} bairros")
except:
    print("Shapefile não encontrado. Faça upload:")
    from google.colab import files
    uploaded = files.upload()
    shapefile_path = list(uploaded.keys())[0]
    gdf_bairros = gpd.read_file(shapefile_path)
    gdf_bairros = gdf_bairros.to_crs("EPSG:4326")

coluna_nome_bairro = 'NM_BAIRROS'  # AJUSTE SE NECESSÁRIO

# Filtrar baixa renda
gdf_filtrado = gdf_bairros[gdf_bairros['MediaBairr'] < 350].copy()
print(f"{len(gdf_filtrado)} bairros de baixa renda")

# ============================================================================
# 4. EXTRAÇÃO NDVI (GEE)
# ============================================================================

def extrair_ndvi_temporal(geom, inicio, fim, nome):
    ee_geom = ee.Geometry.Polygon(list(geom.__geo_interface__['coordinates'][0]))
    colecao = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterDate(inicio, fim)
               .filterBounds(ee_geom)
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

    def calc_ndvi(img):
        return img.normalizedDifference(['B8', 'B4']).rename('NDVI').addBands(img)

    colecao_ndvi = colecao.map(calc_ndvi)

    def extrair(img):
        stats = img.select('NDVI').reduceRegion(ee.Reducer.mean(), ee_geom, 10, bestEffort=True)
        return ee.Feature(None, {
            'data': img.date().format('YYYY-MM-dd'),
            'ndvi': stats.get('NDVI'),
            'area': nome
        })

    try:
        serie = colecao_ndvi.map(extrair).getInfo()['features']
        dados = []
        for f in serie:
            p = f['properties']
            if p['ndvi'] is not None:
                dados.append({'data': pd.to_datetime(p['data']), 'ndvi': p['ndvi'], 'area': p['area']})
        return pd.DataFrame(dados)
    except:
        return pd.DataFrame()

# Período
data_inicio = '2022-01-01'
data_fim = '2024-12-31'

print(f"\nExtraindo NDVI de {len(gdf_filtrado)} bairros...")
lista_series = []
for idx, row in gdf_filtrado.iterrows():
    nome = row.get(coluna_nome_bairro, f'Bairro_{idx}')
    print(f"  [{idx+1}] {nome}...", end="")
    df_temp = extrair_ndvi_temporal(row.geometry, data_inicio, data_fim, nome)
    if len(df_temp) > 0:
        lista_series.append(df_temp)
        print(f" {len(df_temp)} obs")
    else:
        print(" sem dados")

if lista_series:
    df_ndvi = pd.concat(lista_series, ignore_index=True)
    df_ndvi.to_csv('ndvi_serie_temporal.csv', index=False)
    print(f"\nExtração concluída: {len(df_ndvi)} observações")
else:
    raise ValueError("Nenhum dado extraído")

# ============================================================================
# 5. ANÁLISE TEMPORAL
# ============================================================================

df_ndvi['data'] = pd.to_datetime(df_ndvi['data'])
df_ndvi['ano_mes'] = df_ndvi['data'].dt.to_period('M')
df_mensal = df_ndvi.groupby(['area', 'ano_mes']).ndvi.mean().reset_index()
df_mensal['data'] = df_mensal['ano_mes'].dt.to_timestamp()

# Decomposição para TODOS os bairros
os.makedirs('resultados/decomposicao', exist_ok=True)
for bairro in df_mensal['area'].unique():
    df_b = df_mensal[df_mensal['area'] == bairro].set_index('data').sort_index()
    if len(df_b) < 12: continue
    df_b['ndvi'] = df_b['ndvi'].interpolate()
    try:
        decomp = seasonal_decompose(df_b['ndvi'], model='additive', period=12)
        fig, axes = plt.subplots(4,1,figsize=(12,8))
        for i, comp in enumerate(['observed','trend','seasonal','resid']):
            getattr(decomp, comp).plot(ax=axes[i], title=f'{bairro} - {comp.capitalize()}')
        plt.tight_layout()
        plt.savefig(f'resultados/decomposicao/decomp_{bairro}.png', dpi=200)
        plt.close()
    except: pass

# ============================================================================
# 6. ANÁLISE ESPACIAL (LISA)
# ============================================================================

df_ndvi['ano'] = df_ndvi['data'].dt.year
df_anual = df_ndvi.groupby(['area','ano']).ndvi.mean().reset_index()
anos = sorted(df_anual['ano'].unique())

gdfs_temporais = {}
for ano in anos:
    df_ano = df_anual[df_anual['ano'] == ano]
    gdf_ano = gdf_bairros.merge(df_ano, left_on=coluna_nome_bairro, right_on='area', how='inner')
    if len(gdf_ano) > 0:
        gdfs_temporais[ano] = gdf_ano

lisa_resultados = {}
for ano in anos:
    gdf = gdfs_temporais[ano].copy()
    if len(gdf) < 3: continue
    w = Queen.from_dataframe(gdf)
    w.transform = 'r'
    lisa = Moran_Local(gdf['ndvi'].values, w)
    gdf['cluster'] = pd.Series(lisa.q).map({1:'HH',2:'LH',3:'LL',4:'HL'}).fillna('Não Sig.')
    gdf.loc[lisa.p_sim > 0.05, 'cluster'] = 'Não Sig.'
    lisa_resultados[ano] = {'gdf': gdf}

# ============================================================================
# 7. PRIORIDADES
# ============================================================================

ano_atual = anos[-1]
gdf_atual = lisa_resultados[ano_atual]['gdf'].copy()
gdf_atual['prioridade_score'] = 25
gdf_atual.loc[gdf_atual['cluster'] == 'LL', 'prioridade_score'] += 50
mediana = gdf_atual['ndvi'].median()
gdf_atual.loc[gdf_atual['ndvi'] < mediana, 'prioridade_score'] += 25
gdf_atual['prioridade_score'] = gdf_atual['prioridade_score'].clip(upper=100)
bins = [0,40,60,80,100]
gdf_atual['prioridade'] = pd.cut(gdf_atual['prioridade_score'], bins=bins,
                                 labels=['Baixa','Média','Alta','Crítica'], include_lowest=True)

gdf_atual[['area','ndvi','cluster','prioridade_score','prioridade']].to_csv('ranking_prioridades_final.csv', index=False)

# ============================================================================
# 8. ANIMAÇÃO PLOTLY
# ============================================================================

gdf_anim = gdf_bairros.merge(df_anual, left_on=coluna_nome_bairro, right_on='area', how='inner')
gdf_anim = gdf_anim[gdf_anim.geometry.is_valid].copy().to_crs('EPSG:4326')
gdf_anim = gdf_anim.reset_index(drop=True)
gdf_anim['id'] = gdf_anim.index.astype(str)

fig = px.choropleth_mapbox(
    gdf_anim, geojson=gdf_anim.__geo_interface__, locations='id', color='ndvi',
    animation_frame='ano', color_continuous_scale='RdYlGn', range_color=[0,1],
    mapbox_style="open-street-map", zoom=10.5,
    center={"lat": gdf_anim.geometry.centroid.y.mean(), "lon": gdf_anim.geometry.centroid.x.mean()},
    opacity=0.8, title="Evolução Temporal do NDVI"
)
fig.update_layout(height=700, margin=dict(r=0,t=60,l=0,b=0))
fig.show()
fig.write_html('resultados/animacao_ndvi.html')

# ============================================================================
# 9. SALVAR TUDO E DOWNLOAD
# ============================================================================

os.makedirs('resultados', exist_ok=True)
for f in os.listdir('.'):
    if f.endswith(('.csv','.png','.html')):
        os.replace(f, f'resultados/{f}')

!zip -r resultados_tcc.zip resultados/
from google.colab import files
files.download('resultados_tcc.zip')

print("\nANÁLISE CONCLUÍDA! Baixe o ZIP com todos os resultados.")